# Exploring Sentinel-1 over the ACT

This notebook provides an example of the European Space Agency (ESA) Sentinel-1 and Sentinel-2 data. Sentinel-1 is a C-band (wavelength of 5.6 cm) Synthetic Aperture Radar (SAR) instrument with a multi-temporal frequency of 12 days. The SAR data are processed into radar backscatter and dual-polarimetric decomposition (described below). Sentinel-2 is an optical instrument, providing information in the visible and infrared bands. These data are compared to show the different responses in an urban environment, as well as the vegetation dynamics in grasslands.

Sentinel-1 (in this case it is the dual-pol Radar Vegetation Index (RVI) and Entropy bands) and Sentinel-2 NDVI are compared for multitemporal trends due to their relationships with vegetation biomass. 


In [ ]:
import dask
import distributed
from dask_kubernetes import KubeCluster
from dask.distributed import Client,LocalCluster
from dask.distributed import wait, progress
cluster = None
# Click on the 'Dashboard link' to monitor calculation progress
#cluster = KubeCluster()
cluster = LocalCluster()
cluster.scale_up(4)
#cluster.adapt(minimum=1, maximum=3, scale_factor=2, startup_cost="10s", wait_count=5, target_duration="10s")
client = distributed.Client(cluster)

In [ ]:
# Import relevant modules for this Jupyter Notebook and start datacube
import sys
import datacube
import numpy as np
import xarray as xr
from datacube.storage import masking
from datacube.storage.masking import mask_to_dict
#from datacube.utils import geometry
#from datacube.utils.geometry import CRS
from matplotlib import pyplot as plt
from IPython.display import display
from IPython.display import HTML
import ipywidgets as widgets
from ipywidgets import interact
#from matplotlib import pyplot as plt
import matplotlib.image as mplImage
import warnings
warnings.filterwarnings('ignore', module='datacube')

import logging
logging.basicConfig(stream=sys.stdout, level=logging.CRITICAL)
for logger_name in ('boto', 'boto3', 'botocore', 's3transfer', 'rasterio', 'distributed', 'distributed.client'):
    logging.getLogger(logger_name).setLevel(logging.CRITICAL)

dc = datacube.Datacube(env='datacube')

# View area of interest (Canberra)
>  Currently 2019 January to September 2019 around the ACT region <br>

In [ ]:
from ipyleaflet import Map, WMSLayer, LayersControl

Year = '2019'
myloc = 'San Francisco'
query = {

# Canberra
'y': (37.6749, 37.8749),
'x': (-122.5195, -122.3195),

'time': (Year + '-01-01', Year + '-02-01'),
'crs': 'EPSG:4326',
'output_crs': 'EPSG: 3488',
'resolution': (20, -20)
}

m = Map(center=(37.7749, -122.4195), zoom=12, layout=dict(height='600px')) # ACT
m

# Load Sentinel-2 for the same area and compare the two

In [ ]:
dc_S2 = datacube.Datacube(app='s2a_scene')
bands_of_interest = ['B04_20m', 'B03_20m', 'B02_20m','SCL_20m']
#ds = dc_S2.load(product = 's2_l2a_scene', group_by='solar_day', measurements = bands_of_interest, **query, dask_chunks={'time': 1})
#s2_data = ds.load()
#del ds
s2_data = dc_S2.load(product = 's2_l2a_scene', group_by='solar_day', measurements = bands_of_interest, **query, dask_chunks={'time': 1})

s2 = s2_data.rename({'B04_20m': 'red',
                     'B03_20m': 'green',
                     'B02_20m': 'blue',
                     'SCL_20m': 'SCL'})
del s2_data

# Identify images that have minimum nulls and remove them
# Uses code from https://github.com/fangfy/dea-projects/blob/master/water_interoperability/sentinel1_load_and_classify_nci.ipynb
# [mask for clouds [0=no data, 1=saturated, 2=dark pixels, 3=cloud shadow, 4=veg, 5=soil, 
# 6=water, 7, 8, 9=clouds (low/med/high prob), 10=cirrus, 11= snow]]

total_px=s2.dims['x']*s2.dims['y']
valid=s2.where(s2.red!=0).where(s2.SCL<=7).where(s2.SCL>=2).where(s2.SCL!=3).count(dim=('x','y'))

good=(valid.red/total_px)>0.5

s2_good = s2.sel(time=good)
del s2

# replace 0 with nan
s2_good_clean = s2_good.where(s2_good!=0)
del s2_good

from datacube.storage import masking

# Set all nodata pixels to `NaN`:
s2_good_clean = masking.mask_invalid_data(s2_good_clean)
s2_good_clean = s2_good_clean.where((s2_good_clean >= 0) & (s2_good_clean<=4000))

s2_good_clean
s2 = s2_good_clean
del s2_good_clean

print('Sentinel-2 data for ', myloc,': ',s2)

## RGB Sentinel-2 scene (left) and Sentinel-1 VV where Red = March, Green = June, Blue =September (right)
### The build-up areas show as consistently bright in the Sentinel-1 data, while the water is consistently dark. Forested areas have a medium backscatter, while grassland is relatively low.

In [ ]:
# View S1 across multiple dates

Red_date_VV, Green_date_VV, Blue_date_VV = smoothed.vv.isel(time=18), smoothed.vv.isel(time=10), smoothed.vv.isel(time=3)
s1_RGB=Red_date_VV.to_dataset(name='Red_date_VV')
s1_RGB['Green_date_VV'], s1_RGB['Blue_date_VV'] = Green_date_VV, Blue_date_VV
s1_RGB = s1_RGB.assign_attrs(bs_attrs)
#s1_RGB = s1_RGB.load()

# plot RGB images
fix, axes = plt.subplots(ncols=2, figsize=(20,10))

image_array = s2[['red', 'green', 'blue']].isel(time=8).compute().to_array()
image_array.plot.imshow(robust=True, ax=axes[0]);
image_array_s1vv = s1_RGB[['Red_date_VV', 'Green_date_VV', 'Blue_date_VV']].to_array()
image_array_s1vv.plot.imshow(robust=True, vmin=0, vmax=0.25, ax=axes[1], add_labels=False);

plt.tight_layout()
plt.draw()

# View area of interest (Lake George)
>  Currently 2019 January to September 2019 around the ACT region <br>

In [ ]:
from ipyleaflet import Map, WMSLayer, LayersControl

Year = '2019'
myloc = 'ACT'
query = {
    
# Lake George
'y': (-35.15, -35.28),
'x': (149.35, 149.48),

'time': (Year + '-01-01', Year + '-10-01'),
'crs': 'EPSG:4326',
'output_crs': 'EPSG: 3577',
'resolution': (20, -20)
}

m = Map(center=(-35.2, 149.4), zoom=12, layout=dict(height='600px')) # Lake George
m

# Read radar backscatter for area of interest, clean and smooth data

In [ ]:
bs=dc.load(product='s1_intensity_scene', group_by='solar_day', **query, dask_chunks={'time': 1})

# Identify images that have minimum nulls
# Uses code from https://github.com/fangfy/dea-projects/blob/master/water_interoperability/sentinel1_load_and_classify_nci.ipynb

total_px=bs.dims['x']*bs.dims['y']
valid=bs.where(bs.vv!=0).where(bs.vh!=0).count(dim=('x','y'))

good=(valid.vh/total_px)>0.5

bs_good = bs.sel(time=good)
# replace 0 with nan
bs_clean = bs_good.where(bs_good!=0)

# Adapted from https://stackoverflow.com/questions/39785970/speckle-lee-filter-in-python
from scipy.ndimage.filters import uniform_filter
from scipy.ndimage.measurements import variance

def lee_filter(da, size):
    img = da.values
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

# save the nodata mask
nodata_mask = bs_clean.isnull().to_array().any(axis=0)
# Convert backscatter nans to 0 for lee filter
bs_good_zerofilled = bs_good.where(~bs_good.isnull(), 0).compute()

# Apply speckle filter
smoothed_vv=bs_good_zerofilled.vv.groupby('time').apply(lee_filter, size=7)
smoothed_vh=bs_good_zerofilled.vh.groupby('time').apply(lee_filter, size=7)

# Create smoothed dataset with Nans and assign attributes
smoothed=smoothed_vv.to_dataset(name='vv')
smoothed['vh']=smoothed_vh
smoothed=smoothed.where(~nodata_mask)

# Remove unused data
bs_attrs = bs.attrs
smoothed = smoothed.assign_attrs(bs_attrs)

del bs, bs_good, bs_clean
print('Backscatter data for ', myloc,': ',smoothed)

## View smoothed images (VV or VH)

The VV & VH images show the radar backscattered from the imaged area when: transmitted in the vertical direction and received in the vertical direction (VV); or transmitted in the vertical direction and received in the horizontal direction (VH).

Built up areas appear as bright in all images due to strong double-bounce scattering. Surface water appears as dark due to specular scattering. Forested areas have a medium backscatter intensity due to a combination of double-bounce scattering from the ground and tree trunk, and volume scattering within the canopy. Open pastures have a low-medium backscatter intensity depending on the level of vegetation biomass and soil moisture.

VV is better at discriminating build-up areas from forest/pasture areas. VH is better related to vegetation biomass.


In [ ]:
# View VV and VH scenes
ntimes=len(smoothed.time.values)
viz = smoothed.compute()
viz.vv.isel(time=slice(0,ntimes,4)).plot(col='time',col_wrap=6, vmin=0, vmax=0.18, figsize=(20,3.5));
viz.vh.isel(time=slice(0,ntimes,4)).plot(col='time',col_wrap=6, vmin=0, vmax=0.05, figsize=(20,3.5));

# Load Sentinel-2 for the same area and show (mostly) cloud-free image

In [ ]:
dc_S2 = datacube.Datacube(app='s2a_scene')
bands_of_interest = ['B04_20m', 'B03_20m', 'B02_20m', 'B8A_20m','SCL_20m']
s2_data = dc_S2.load(product = 's2_l2a_scene', group_by='solar_day', measurements = bands_of_interest, **query, dask_chunks={'time': 1})
#s2_data = ds.load() # Force in memory load
#del ds

s2 = s2_data.rename({'B04_20m': 'red',
                     'B03_20m': 'green',
                     'B02_20m': 'blue',
                     'B8A_20m': 'nir',
                     'SCL_20m': 'SCL'})
del s2_data

# Identify images that have minimum nulls and remove them
# Uses code from https://github.com/fangfy/dea-projects/blob/master/water_interoperability/sentinel1_load_and_classify_nci.ipynb
# [mask for clouds [0=no data, 1=saturated, 2=dark pixels, 3=cloud shadow, 4=veg, 5=soil, 
# 6=water, 7, 8, 9=clouds (low/med/high prob), 10=cirrus, 11= snow]]

total_px=s2.dims['x']*s2.dims['y']
valid=s2.where(s2.red!=0).where(s2.SCL<=7).where(s2.SCL>=2).where(s2.SCL!=3).count(dim=('x','y'))

good=(valid.red/total_px)>0.5

s2_good = s2.sel(time=good)
del s2

# replace 0 with nan
s2_good_clean = s2_good.where(s2_good!=0)
del s2_good

from datacube.storage import masking

# Set all nodata pixels to `NaN`:
s2_good_clean = masking.mask_invalid_data(s2_good_clean)
s2_good_clean = s2_good_clean.where((s2_good_clean >= 0) & (s2_good_clean<=4000))

s2_good_clean
s2 = s2_good_clean
#s2.persist()
del s2_good_clean
print('Sentinel-2 data for ', myloc,': ',s2)

## Sentinel-2 RGB (left) and multi-date Sentinel-1 image for VH (right)

RGB = March, June, September

Built-up areas show as bright white. Agricultural/irrigated fields show in different colours depending on time of year. Forest shows as grey/white.

In [ ]:
fix, axes = plt.subplots(ncols=2, figsize=(20,9))

# View S2 RGB and S1 across multiple dates
Red_date_VH, Green_date_VH, Blue_date_VH = smoothed.vh.isel(time=19), smoothed.vh.isel(time=10), smoothed.vh.isel(time=3)
s1_RGB=Red_date_VH.to_dataset(name='Red_date_VH')
s1_RGB['Green_date_VH'], s1_RGB['Blue_date_VH'] = Green_date_VH, Blue_date_VH
s1_RGB = s1_RGB.assign_attrs(bs_attrs)
#s1_RGB = s1_RGB.load()

image_array = s2[['red', 'green', 'blue']].isel(time=8).compute().to_array()
image_array.plot.imshow(robust=True, ax=axes[0]);
image_array_s1vh = s1_RGB[['Red_date_VH', 'Green_date_VH', 'Blue_date_VH']].to_array()
image_array_s1vh.plot.imshow(robust=True, vmin=0, vmax=0.04, ax=axes[1], add_labels=False);

plt.tight_layout()
plt.draw()

## Calculate and view modified dual-pol Radar Vegetation Index images

The Radar Vegetation Index (RVI) can be used to look at change in vegetation biomass through time. It is an index based on the VV and VH bands, so is less affected by multi-temporal variation in soil moisture compared to the intensity bands. In this case, the C-band wavelength of Sentinel-1 can show changes in the biomass of grasslands rather than forests.


In [ ]:
# View RVI data

smoothed['RVI'] = 4*smoothed.vh/(smoothed.vv + smoothed.vh)

ntimes=len(smoothed.time.values)
viz = smoothed.RVI.compute()
viz.isel(time=slice(0,ntimes,4)).plot(col='time',col_wrap=6, vmin=0.2, vmax=1.5, figsize=(20,3.5));

# Read in dual-pol decomposition data

Dual-polarimetric decomposition helps to describe the scattering behaviour of the radar wave as it interacts with the features in the landscape. The Entropy and Alpha bands can help discriminate forest from non-forest and highlight complex scattering behaviour in urban environments. The Entropy band is also related to vegetation biomass in some circumstances.

In [ ]:
#ds=dc.load(product='s1_decomposition_scene', group_by='solar_day', **query, dask_chunks={'time': 1})
#dp = ds.load()
#del ds
dp=dc.load(product='s1_decomposition_scene', group_by='solar_day', **query, dask_chunks={'time': 1})

dp = dp.where(dp.entropy!=0)

# Identify images that have minimum nulls
# Uses code from https://github.com/fangfy/dea-projects/blob/master/water_interoperability/sentinel1_load_and_classify_nci.ipynb

total_px=len(dp.x)*len(dp.y)
valid=dp.where(dp.entropy!=0).where(dp.anisotropy!=0).count(dim=('x','y'))

good=(valid.entropy/total_px)>0.5

dp = dp.sel(time=good)
# replace 0 with nan
dp = dp.where(dp!=0)
print('Dual polarimetric decomposition data for ', myloc,': ',dp)

## View Alpha or Entropy images

In [ ]:
# View alpha and entropy bands
ntimes=len(dp.time.values)
viz = dp.compute()
viz.alpha.isel(time=slice(0,ntimes,4)).plot(col='time',col_wrap=6, vmin=40, vmax=80.0, figsize=(20,3.5));
viz.entropy.isel(time=slice(0,ntimes,4)).plot(col='time',col_wrap=6, vmin=0, vmax=1.0, figsize=(20,3.5));

## Extract NDVI time series from Sentinel-2

In [ ]:
# Create NDVI band
s2['ndvi']=(s2.nir - s2.red)/(s2.nir + s2.red)

# Remove any remaining erroneous values (where NDVI <-1.0 and NDVI > 1.0)
#s2['ndvi'] = s2.ndvi.where(s2.ndvi > -1.0).where(s2.ndvi < 1.0)
viz = s2['ndvi'].compute() # Force in memory copy for viz
# View S2 NDVI time series
ntimes=len(s2.ndvi.time.values)
viz.plot(cmap='viridis', col='time', col_wrap=5,figsize=(15,20));

# Look at RVI, Entropy and NDVI time series over different landcover types

## Plots of monthly time series based on selected pixels. Different temporal trends can help distinguish different landcover types

### Pasture within Lake George

Very similar temporal trends from RVI, Entropy and NDVI

In [ ]:
# plot RVI, Entropy and NDVI through time based on the x,y point selected

import ipywidgets as widgets
from ipywidgets import interact
warnings.filterwarnings('ignore')

# expand selected x,y pixel to make a square area of interest
#pixelx, pixely = 1577500, -3955000 # Ag field north of town
#pixelx, pixely = 1575000, -3949000 # Forest
pixelx, pixely = 1579000, -3948000 # Lake George pasture
#pixelx, pixely = 1577100, -3960000 # Irrigated crop

xp, yp=slice(pixelx+100,pixelx-100), slice(pixely-100,pixely+100)

fix, axes = plt.subplots(ncols=3, figsize=(18,5))

(smoothed.sel(x=xp,y=yp).RVI.groupby('time.month').mean()).plot(color='r', ax=axes[0]);
(dp.sel(x=xp,y=yp).entropy.groupby('time.month').mean()).plot(color='b', ax=axes[1]);
(s2.sel(x=xp,y=yp).ndvi.groupby('time.month').mean()).plot(color='g', ax=axes[2]);

plt.tight_layout()
plt.draw()

### Irrigated crop

Different temporal trend between RVI, Entropy and NDVI

In [ ]:
# plot RVI, Entropy and NDVI through time based on the x,y point selected

import ipywidgets as widgets
from ipywidgets import interact
warnings.filterwarnings('ignore')

# expand selected x,y pixel to make a square area of interest
pixelx, pixely = 1577100, -3960000 # Irrigated crop

xp, yp=slice(pixelx+100,pixelx-100), slice(pixely-100,pixely+100)

fix, axes = plt.subplots(ncols=3, figsize=(18,5))

(smoothed.sel(x=xp,y=yp).RVI.groupby('time.month').mean()).plot(color='r', ax=axes[0]);
(dp.sel(x=xp,y=yp).entropy.groupby('time.month').mean()).plot(color='b', ax=axes[1]);
(s2.sel(x=xp,y=yp).ndvi.groupby('time.month').mean()).plot(color='g', ax=axes[2]);

plt.tight_layout()
plt.draw()

## Forest

Different temporal trends between RVI, Entropy and NDVI (although remnant cloud shadow reduces the NDVI for July)

In [ ]:
# plot RVI, Entropy and NDVI through time based on the x,y point selected

import ipywidgets as widgets
from ipywidgets import interact
warnings.filterwarnings('ignore')

# expand selected x,y pixel to make a square area of interest
pixelx, pixely = 1575000, -3949000 # Forest

xp, yp=slice(pixelx+100,pixelx-100), slice(pixely-100,pixely+100)

fix, axes = plt.subplots(ncols=3, figsize=(18,5))

(smoothed.sel(x=xp,y=yp).RVI.groupby('time.month').mean()).plot(color='r', ax=axes[0]);
(dp.sel(x=xp,y=yp).entropy.groupby('time.month').mean()).plot(color='b', ax=axes[1]);
(s2.sel(x=xp,y=yp).ndvi.groupby('time.month').mean()).plot(color='g', ax=axes[2]);

plt.tight_layout()
plt.draw()

# Show dynamics Statistics
## Maximum multi-temporal difference in RVI, Entropy and NDVI

The pasture along the eastern side of Lake George showed greatest variation in RVI and Entropy. Forested areas varied little with Entropy. 

In [ ]:
# Calculate annual range (max minus min) for radar backscatter RVI, dul-pol decomposition Entropy, and NDVI
from matplotlib import pyplot as plt
#pylab.rcParams['font.size']=7

smoothed['RVI_range'] = smoothed.RVI.max(dim='time') - smoothed.RVI.min(dim='time')
dp['entropy_range'] = dp.entropy.max(dim='time') - dp.entropy.min(dim='time')
s2['ndvi_range'] = s2.ndvi.max(dim='time') - s2.ndvi.min(dim='time')

fix, axes = plt.subplots(ncols=3, figsize=(15,4))

smoothed.RVI_range.plot(vmin=0.2,vmax=1.2, ax=axes[0]);
dp.entropy_range.plot(vmin=0.1,vmax=0.5, ax=axes[1]);
s2.ndvi_range.plot(vmin=0.0,vmax=1.0, ax=axes[2]);
plt.tight_layout()
plt.draw()

## Annual (nearly) RVI, Entropy and NDVI mean, min, max and std

In [ ]:
# Submit the tasks using Dask futures - will run in parallel if they run for any appreciable amount of time.
smoothed_RVI_mean = client.submit(smoothed.RVI.mean, dim='time')
smoothed_RVI_min = client.submit(smoothed.RVI.min, dim='time')
smoothed_RVI_max = client.submit(smoothed.RVI.max, dim='time')
smoothed_RVI_std = client.submit(smoothed.RVI.std, dim='time')
dp_entropy_mean = client.submit(dp.entropy.mean, dim='time')
dp_entropy_min = client.submit(dp.entropy.min, dim='time')
dp_entropy_max = client.submit(dp.entropy.max, dim='time')
dp_entropy_std = client.submit(dp.entropy.std, dim='time')
s2_ndvi_mean = client.submit(s2.ndvi.mean, dim='time')
s2_ndvi_min = client.submit(s2.ndvi.min, dim='time')
s2_ndvi_max = client.submit(s2.ndvi.max, dim='time')
s2_ndvi_std = client.submit(s2.ndvi.std, dim='time')



In [ ]:
# Show RVI, Entropy and NDVI mean, min, max and stand deviation for current year

fig, axes = plt.subplots(ncols=4, nrows=3, figsize=(15,8))

smoothed_RVI_mean.result().plot(vmin=0.0,vmax=1.0, ax=axes[0,0]);
smoothed_RVI_min.result().plot(vmin=0.0,vmax=1.0, ax=axes[0,1]);
smoothed_RVI_max.result().plot(vmin=0.0,vmax=2.0, ax=axes[0,2]);
smoothed_RVI_std.result().plot(vmin=0.0,vmax=0.3, ax=axes[0,3]);
dp_entropy_mean.result().plot(vmin=0.2,vmax=0.9, ax=axes[1,0]);
dp_entropy_min.result().plot(vmin=0.0,vmax=0.8, ax=axes[1,1]);
dp_entropy_max.result().plot(vmin=0.2,vmax=1.0, ax=axes[1,2]);
dp_entropy_std.result().plot(vmin=0.05,vmax=0.15, ax=axes[1,3]);
s2_ndvi_mean.result().plot(vmin=0.1,vmax=0.6, ax=axes[2,0]);
s2_ndvi_min.result().plot(vmin=0.0,vmax=0.5, ax=axes[2,1]);
s2_ndvi_max.result().plot(vmin=0.0,vmax=1.0, ax=axes[2,2]);
s2_ndvi_std.result().plot(vmin=0.0,vmax=0.2, ax=axes[2,3]);

axes[0,0].set_title('Mean')
axes[0,1].set_title('Min')
axes[0,2].set_title('Max')
axes[0,3].set_title('Std')
plt.tight_layout()
plt.draw()

In [ ]:
client.close()
client = None
cluster.close()
cluster = None